# Rozwiązanie laboratorium 2

In [36]:
from queue import PriorityQueue
from time import perf_counter

Funkcje konwertujące

In [37]:
def string_to_int(string):
    val = 0
    for x in string:
        val = val << 1 | (0 if x == '0' else 1)
    return val


def int_to_string(x, no_of_bits):
    string = []
    mask = 1 << (no_of_bits - 1)
    for _ in range(no_of_bits):
        string.append("1" if x & mask else "0")
        mask >>= 1
    return "".join(string)

Wczytytywanie i odczytywanie z plików binarnych/tekstowych

In [15]:
def read_file_to_string(filename):
    with open(filename, "r", encoding="UTF-8") as f:
        data = f.read()
    return data


def read_binary_file_to_string(filename):
    with open(filename, 'rb') as f:
        bit_data = f.read()

    data = []
    for bit in bit_data[:-2]:
        data.append(int_to_string(bit, 8))
    last = ""
    mask = 1 << 7
    if bit_data[-1]:
        for j in range(bit_data[-1]):
            last += "1" if mask & bit_data[-2] else "0"
            mask >>= 1
    else:
        for j in range(8):
            last += "1" if mask & bit_data[-2] else "0"
            mask >>= 1
    data.append(last)
    return "".join(data)


def write_string_to_binary_file(filename, text):
    b = bytearray()
    for i in range(0, len(text), 8):
        b.append(string_to_int(text[i:i+8]))

    with open(filename, 'wb') as f:
        f.write(b)

In [16]:
def add_last_bits(text):
    padding_length = len(text) % 8
    text += "0" * ((8 - padding_length)%8)
    text += int_to_string(padding_length, 8)
    return text

### Statyczne drzewo Huffmana

Przy kompresowaniu pliku początkowo zakodowane litery zapisuje jako zera i jedynki (znaki ascii). Też do takiej formy
zapisuje wczytywany plik. Dopiero potem znaki odpowiednio konweruje na liczby i zapisuje do plików. Używanie stringów było jednak najwygodniejsze.

In [17]:
class StaticNode:
    def __init__(self, character=None):
        self.character = character
        self.left = None
        self.right = None

class StaticHuffman:
    def __init__(self):
        self.tree_root = None
        self.frequency_dict = {}
        self.code_dict = {}

    def build_frequency_dict(self, data):
        for c in data:
            if c not in self.frequency_dict:
                self.frequency_dict[c] = 1
            else:
                self.frequency_dict[c] += 1

    def build_tree(self):
        pq = PriorityQueue()
        for c in self.frequency_dict:
            pq.put((self.frequency_dict[c], c, StaticNode(c)))

        while True:
            freq1, str1, node1 = pq.get()
            if pq.empty():
                self.tree_root = node1
                return
            freq2, str2, node2 = pq.get()
            new_node = StaticNode()
            new_node.left = node1
            new_node.right = node2
            pq.put((freq1 + freq2, str1 + str2, new_node))

    def code_characters(self):
        def traverse_tree(node, code=""):
            if node.character is not None:
                self.code_dict[node.character] = code
            else:
                traverse_tree(node.left, code + '0')
                traverse_tree(node.right, code + '1')

        traverse_tree(self.tree_root, code="")

    def encode_text(self, text):
        self.build_frequency_dict(text)
        self.build_tree()
        self.code_characters()
        encoded = []
        for c in text:
            encoded.append(self.code_dict[c])
        return add_last_bits("".join(encoded))

    def decode_text(self, text):
        ind = 0
        decoded = []
        while ind < len(text):
            ptr = self.tree_root
            while ptr.character is None:
                ptr = ptr.left if text[ind] == '0' else ptr.right
                ind += 1
            decoded.append(ptr.character)
        return "".join(decoded)

### Dynamiczne drzewo Huffmana

Podobnie jak w statycznym drzewie Huffmana w samym programie używam stringów.
Moja implementacja bazuje na algorytmie FGK(Faller-Gallager-Knuth). Otrzymana struktura nie jest najszybsza.
Można by to poprawić, gdyby struktury self.node_weights i self.leaf_weights ( atrybuty klasy AdaptiveHuffman ), byłyby listami posortowanych
zbiorór (np. drzew AVL), a nie zwykłych zbiorów.
Ponadto, jako że przy dekodowaniu w dynamicznym drzewie Huffmana drzewo jest tworzone na nowo i nie wiadomo jakie litery mogą się pojawić, w momencie, gdy
po raz pierwszy napotykamy musimy ją zapisać w kodzie "normalnie". W pythonie jest dostępna funkcja ord(), która dla znaku zwraca liczbę z kodu ascii
Działa ona jednak dla każdej znaku z UTF-8 ( większość plików chociażby z gutenberg.ord jest w takim formacie). Niektóre jednak znaki nie mieszczą się na jednym bajcie, lecz dopiero na dwóch. Dlatego przy kodowaniu nowo napotkanego znaku używałem zawsze 16 bitów. Dany znak jest tak zapisywany tylko raz, więc nie ma to znaczenia dla plików, które są większe niż kilka kilobajtów, lecz dla małych plików, w których jest dużo różnych znaków współczynnik kompresji może być bardzo niski.

In [23]:
class AdaptiveNode:
    def __init__(self, index, weight, character, external):
        self.index = index
        self.weight = weight
        self.character = character
        self.external = external
        self.left = None
        self.right = None
        self.parent = None


def interchange(node, change):
    if change != node:
        change.index, node.index = node.index, change.index
        parent_change, parent_node = change.parent, node.parent
        if parent_change.left == change:
            if parent_node.left == node:
                parent_change.left, parent_node.left = node, change
            else:
                parent_change.left, parent_node.right = node, change
        else:
            if parent_node.left == node:
                parent_change.right, parent_node.left = node, change
            else:
                parent_change.right, parent_node.right = node, change
        node.parent, change.parent = parent_change, parent_node


def update_weight(node, node_dict):
    node_dict[node.weight].remove(node)
    if node.weight + 1 >= len(node_dict):
        node_dict.append(set())
    node_dict[node.weight + 1].add(node)


class AdaptiveHuffman:
    def __init__(self):
        self.root = AdaptiveNode(1000, 0, "NYT", True)
        self.NYT = self.root
        self.free_index = 999
        self.leaves = {}
        self.node_weights = [set(), set()]
        self.leaf_weights = [set(), set()]

    def get_leaf_code(self, node):
        code = []
        while node != self.root:
            code.append("0" if node == node.parent.left else "1")
            node = node.parent
        return "".join(code)[::-1]

    def add_new_node(self, char):
        right_child = AdaptiveNode(self.free_index, 1, char, True)
        self.free_index -= 1
        left_child = AdaptiveNode(self.free_index, 0, "NYT", True)
        self.free_index -= 1

        internal, self.NYT = self.NYT, left_child

        internal.weight = 1
        internal.character = ""
        internal.external = False
        internal.left = left_child
        internal.right = right_child

        right_child.parent = internal
        left_child.parent = internal

        self.leaves[char] = right_child
        self.node_weights[1].add(right_child)
        if internal != self.root:
            self.node_weights[1].add(internal)

        self.leaf_weights[1].add(right_child)
        self.update(internal)

    def update(self, node):
        while node != self.root:
            if node.parent.left == self.NYT:
                change = max(self.leaf_weights[node.weight], key=lambda item: item.index)
                interchange(node, change)
            else:
                change = max(self.node_weights[node.weight], key=lambda item: item.index)
                interchange(node, change)

            if node.external:
                update_weight(node, self.leaf_weights)
            update_weight(node, self.node_weights)
            node.weight += 1
            node = node.parent

        self.root.weight += 1

    def encode_text(self, text):
        encoded = []
        for c in text:
            if c not in self.leaves:
                encoded.append(self.get_leaf_code(self.NYT))
                encoded.append(int_to_string(ord(c), 16))
                self.add_new_node(c)
            else:
                encoded.append(self.get_leaf_code(self.leaves[c]))
                self.update(self.leaves[c])

        return add_last_bits("".join(encoded))


    def decode_text(self, text):
        decoded = []
        ind = 0
        while ind < len(text):
            ptr = self.root
            while not ptr.external:
                ptr = ptr.left if text[ind] == '0' else ptr.right
                ind += 1

            if ptr == self.NYT:
                new_char = chr(string_to_int(text[ind:ind+16]))
                decoded.append(new_char)
                self.add_new_node(new_char)
                ind += 16
            else:
                decoded.append(ptr.character)
                self.update(ptr)
        return "".join(decoded)

#### Współczynniki kompresji i testy czasowe

In [61]:
from os import listdir
from os.path import isfile, join
from os.path import getsize

files1 = ["Gutenberg_files/" + f for f in listdir("Gutenberg_files") if isfile(join("Gutenberg_files", f))]
files2 = ["random_files/" + f for f in listdir("random_files") if isfile(join("random_files", f))]
files3 = ["Linux_kernel/" + f for f in listdir("Linux_kernel") if isfile(join("Linux_kernel", f))]
files = files1 + files2 + files3

In [62]:
for file in files:
    static_tree = StaticHuffman()
    adaptive_encoder = AdaptiveHuffman()
    adaptive_decoder = AdaptiveHuffman()
    print("----------------")
    print(f"Plik: {file} o rozmiarze {getsize(file)} bajtów")
    f_content = read_file_to_string(file)
    t = perf_counter()
    compressed = static_tree.encode_text(f_content)
    t = perf_counter() - t
    print("Statyczny Huffman")
    print(f"Czas kompresji {t:.2f}")

    write_string_to_binary_file("compressed.bin", compressed)
    to_decode = read_binary_file_to_string("compressed.bin")
    t = perf_counter()
    decompressed = static_tree.decode_text(to_decode)
    t = perf_counter() - t
    print(f"Czas dekompresji {t:.2f}")
    print(f"Sprawdzenie poprawności kompresji i dekompresji: {decompressed == f_content}")
    print(f"Współczynnik kompresji = {1 - (getsize('compressed.bin') / getsize(file))}")

    print()
    print()

    t = perf_counter()
    compressed = adaptive_encoder.encode_text(f_content)
    t = perf_counter() - t
    print("Dynamiczny Huffman")
    print(f"Czas kompresji {t:.2f}")

    write_string_to_binary_file("compressed.bin", compressed)
    to_decode = read_binary_file_to_string("compressed.bin")

    t = perf_counter()
    decompressed = adaptive_decoder.decode_text(to_decode)
    t = perf_counter() - t
    print(f"Czas dekompresji {t:.2f}")
    print(f"Sprawdzenie poprawności kompresji i dekompresji: {decompressed == f_content}")
    print(f"Współczynnik kompresji = {1 - (getsize('compressed.bin') / getsize(file))}")
    print("\n\n")





----------------
Plik: Gutenberg_files/book.txt o rozmiarze 584 bajtów
Statyczny Huffman
Czas kompresji 0.00
Czas dekompresji 0.00
Sprawdzenie poprawności kompresji i dekompresji: True
Współczynnik kompresji = 0.4486301369863014


Dynamiczny Huffman
Czas kompresji 0.01
Czas dekompresji 0.00
Sprawdzenie poprawności kompresji i dekompresji: True
Współczynnik kompresji = 0.2996575342465754



----------------
Plik: Gutenberg_files/mickiewicz.txt o rozmiarze 14404 bajtów
Statyczny Huffman
Czas kompresji 0.00
Czas dekompresji 0.01
Sprawdzenie poprawności kompresji i dekompresji: True
Współczynnik kompresji = 0.4321021938350458


Dynamiczny Huffman
Czas kompresji 0.13
Czas dekompresji 0.10
Sprawdzenie poprawności kompresji i dekompresji: True
Współczynnik kompresji = 0.4159261316301027



----------------
Plik: Gutenberg_files/moby_dick.txt o rozmiarze 1276229 bajtów
Statyczny Huffman
Czas kompresji 0.62
Czas dekompresji 0.77
Sprawdzenie poprawności kompresji i dekompresji: True
Współczynnik

### Algorytm o zmiennym bloku kompresji LZW (Lempel–Ziv–Welch)

In [257]:
content = read_file_to_string("test")

In [258]:
class TrieNode:
    def __init__(self, prefix_id):
        self.id = prefix_id
        self.edges = {}

In [304]:
def encoding(text):
    encoded = []
    root = TrieNode(0)
    for i in range(1, 256):
        root.edges[chr(i)] = TrieNode(i)
    last = 256
    ptr = root
    for x in text:
        if x not in ptr.edges:
            encoded.append(ptr.id)
            ptr.edges[x] = TrieNode(last)
            last += 1
            ptr = root
        else:
            ptr = ptr.edges[x]

    if ptr != root:
        encoded.append(ptr.id)
    return encoded


def decoding(text):
    decode = []
    words = {}
    for i in range(1, 256):
        words[chr(i)] = chr(i)
    words[""] = ""
    last = 256
    word = ""
    for x in text:
        if x not in words:
            decode.append(x)
            words[word + x] = last
            last += 1
            word = ""
        else:
            decode.append(words[x])
            word = words[word] + words[x]
        print(decode)
        print(words['s'])


    return "".join(decode)


In [305]:
print(len(content))
print(content)
coded = encoding(content)
print(len(coded))
print(coded)
# decoded = decoding(coded)

# print(decoded)


161
siajdasikdklakslanklsdnkasdk;fmak;sfmkmasd
pasjasdjikansdnjladjslnajldjlnajlsdlnad
iasdjklnasljkdlnaslkdnlajkdjsnaldsljnad
oasjdiasikdnmaiksdklnaksldnkladslnasdn
66
[115, 97, 100, 256, 100, 108, 107, 261, 107, 115, 110, 97, 260, 102, 97, 59, 269, 109, 265, 112, 115, 267, 106, 107, 110, 100, 106, 97, 106, 108, 257, 100, 285, 282, 100, 110, 100, 105, 265, 264, 267, 106, 290, 296, 281, 257, 287, 291, 100, 108, 291, 10, 267, 100, 267, 107, 110, 97, 262, 295, 262, 281, 261, 115, 291, 281]
